### Imports and Loading of buildings_metadata.xlsx to a dataframe

In [134]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load the buildings_metadata.xlsx file into a pandas DataFrame
file_path = "buildings_metadata.xlsx"
buildings_metadata_df = pd.read_excel(file_path)

In [135]:
#print all the unique values in the bldg_stories_above field in the dataframe
print(buildings_metadata_df['bldg_stories_above'].unique())

['21 or more' '1' '6 to 10' '2 to 5' nan '11 to 15' '16 to 20']


---

### Plotting Stories vs GWP Intensity

#### Cleaning Data

In [136]:
# Remove any rows from the DataFrame where the bldg_stories_above field is NaN
filtered_bldgs_df1 = buildings_metadata_df.copy()
filtered_bldgs_df1 = filtered_bldgs_df1.dropna(subset=['bldg_stories_above'])

# Remove data points where the "eci_a_to_c_cfa" field is above 1200
filtered_bldgs_df1 = filtered_bldgs_df1[filtered_bldgs_df1['eci_a_to_c_cfa'] <= 1200]

# Limit to New Construction
filtered_bldgs_df1 = filtered_bldgs_df1[filtered_bldgs_df1['bldg_proj_type'] == 'New Construction']

# Calculate the count of data points in each bucket
stories_order = ['1', '2 to 5', '6 to 10', '11 to 15', '16 to 20', '21 or more']
bucket_counts = filtered_bldgs_df1['bldg_stories_above'].value_counts()
bucket_counts = bucket_counts.reindex(stories_order)  # Ensure correct order

#### Plotting Data

In [137]:

# Create a box and whisker plot for the "eci_a_to_c_cfa" field grouped by "bldg_stories_above"
fig1 = px.box(
    filtered_bldgs_df1,
    x='bldg_stories_above',
    y='eci_a_to_c_cfa',
    title='Embodied Carbon Intensities Grouped by Building Stories',
    labels={
        'bldg_stories_above': 'Building Stories Above Ground',
        'eci_a_to_c_cfa': 'Normalized Embodied Carbon Intensity (CFA)',
        'lca_phys_scope': 'LCA Physical Scope',
        'lca_software': 'LCA Software'
    },
    category_orders={'bldg_stories_above': stories_order},  # Ensure correct order
    hover_data=['lca_phys_scope', 'lca_software']  # Include lca_phys_scope in hover data
)

# Add all data points as dots with slight transparency and a different color
fig1.update_traces(
    boxpoints='all',
    jitter=0.3,
    pointpos=0,
    marker=dict(color='rgba(28, 144, 153, 0.5)'),
    fillcolor='rgb(236,226,240)',
    line=dict(color='rgb(166,189,219)')
)

# Update the y-axis to limit the max value and relabel it
fig1.update_yaxes(
    title_text='Normalized Embodied Carbon Intensity (CFA)',
    showgrid=True,
    gridcolor='lightgrey'
)

# Update the x-axis to include the counts in the labels
x_labels_with_counts = [f"{bucket} [{int(count)}]" for bucket, count in bucket_counts.items()]

fig1.update_xaxes(
    ticktext=x_labels_with_counts,  # Use the updated labels with counts
    tickvals=stories_order # Match the original categories
)


# Update the title
fig1.update_layout(
    title={
        'text': 'Embodied Carbon Intensities Grouped by Building Stories',
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 16, 'family': 'Arial', 'color': 'black', 'weight': 'bold'}
    },
    plot_bgcolor='white',
    paper_bgcolor='white',
    autosize=True,
    height=600,
    width=800,
    xaxis=dict(scaleanchor="y"),  # Lock the aspect ratio between x and y axes
)

# Show the plot
fig1.show()

print(bucket_counts)


bldg_stories_above
1              49
2 to 5        117
6 to 10        41
11 to 15       19
16 to 20        7
21 or more      7
Name: count, dtype: int64


---

### Plotting Building Height vs Carbon Intensity

#### Cleaning & Filtering Data

In [138]:
def remove_outliers(df, col_names):
    """
    Remove extreme outliers based on IQR method
    """
    q1 = df[col_names].quantile(0.25)
    q3 = df[col_names].quantile(0.75)
    iqr = q3 - q1

    df = df[~((df[col_names] < (q1 - 1.5 * iqr)) |(df[col_names] > (q3 + 1.5 * iqr))).any(axis=1)]

    return df

In [139]:
# Make a copy of the original dataframe and remove rows with nulls in the "bldg_height" field
filtered_bldgs_df2 = buildings_metadata_df.copy()
filtered_bldgs_df2 = filtered_bldgs_df2.dropna(subset=['bldg_height'])

# Remove outliers based on 'eci_a_to_c_cfa'
filtered_bldgs_df2 = remove_outliers(filtered_bldgs_df2, ['eci_a_to_c_cfa'])

# Limit to New Construction
filtered_bldgs_df2 = filtered_bldgs_df2[filtered_bldgs_df2['bldg_proj_type'] == 'New Construction']

# Calculate the count of data points in each bucket
height_order = ['0-7.5 m', '7.6-15 m', '15.1-22.5 m', '22.6-30 m', '31-45 m', '46-60 m', '61-90 m', 'Over 90 m']
bucket_counts2 = filtered_bldgs_df2['bldg_height'].value_counts()
bucket_counts2 = bucket_counts2.reindex(height_order)  # Ensure correct order

#### Plotting Data

In [166]:
# Create a box and whisker plot for the "eci_a_to_c_cfa" field grouped by "bldg_height"
fig2 = px.box(
    filtered_bldgs_df2,
    x='bldg_height',  # Group by building height
    y='eci_a_to_c_cfa',  # Plot this field
    title='Embodied Carbon Intensities Grouped by Building Height',
    labels={
        'bldg_height': 'Building Height',
        'eci_a_to_c_cfa': 'Normalized Embodied Carbon Intensity (CFA)',
        'lca_phys_scope': 'LCA Physical Scope',
        'lca_software': 'LCA Software'
    },
    category_orders={'bldg_height': height_order},  # Ensure correct order of height buckets
    hover_data=['lca_phys_scope', 'lca_software']  # Include lca_phys_scope and software in hover data
)

# Add all data points as dots with slight transparency and specify color
fig2.update_traces(
    boxpoints='all',
    jitter=0.3,
    pointpos=0,
    marker=dict(color='rgba(28, 144, 153, 0.5)'),
    fillcolor='rgb(236,226,240)',
    line=dict(color='rgb(166,189,219)')
)


fig2.update_yaxes(
    title_text='Normalized Embodied Carbon Intensity (CFA) [kgCO2e/m²]',
    showgrid=True,
    gridcolor='lightgrey'
)

# Update the x-axis to include the counts in the labels
x_labels_with_counts2 = [f"{bucket} [{int(count)}]" for bucket, count in bucket_counts2.items()]

fig2.update_xaxes(
    ticktext=x_labels_with_counts2,  # Use the updated labels with counts
    tickvals=height_order # Match the original categories
)


fig2.update_layout(
    title={
        'text': 'Embodied Carbon Intensities Grouped by Building Height',
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 16, 'family': 'Arial', 'color': 'black', 'weight': 'bold'}
    },
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=700,
    width=800
)

# Show the plot
fig2.show()

---

### Exploring Difference in Intensity Measurements

#### Cleaning and Filtering Data

In [141]:
# Make a copy of the original dataframe and remove rows with nulls in the "bldg_height" field
filtered_bldgs_df3 = buildings_metadata_df.copy()

# Limit to New Construction
filtered_bldgs_df3 = filtered_bldgs_df3[filtered_bldgs_df3['bldg_proj_type'] == 'New Construction']
print(len(filtered_bldgs_df3))

# Limit to buildings without Parking (whether or not there is parking can heavily sway the carbon intensity)
filtered_bldgs_df3 = filtered_bldgs_df3[filtered_bldgs_df3['bldg_park_type'] == 'No Parking']
print(len(filtered_bldgs_df3))

#NOTE Decided to turn this filter off since there weren't that many buildings with this use type
# Remove buildings where the bldg_prim_use is 'Warehouse and Storage' (they might heavily skew the data since they obviously have much fewer occupants)
# filtered_bldgs_df3 = filtered_bldgs_df3[filtered_bldgs_df3['bldg_prim_use'] != 'Warehouse and Storage']
# print(len(filtered_bldgs_df3))

# Remove rows where any of the specified fields have null values
filtered_bldgs_df3 = filtered_bldgs_df3.dropna(subset=['ec_per_occupant_a_to_c', 'eci_a_to_c_gfa'])
print(len(filtered_bldgs_df3))

#Remove extreme outliers based on IQR method for the specified fields
filtered_bldgs_df3 = remove_outliers(filtered_bldgs_df3, ['ec_per_occupant_a_to_c', 'eci_a_to_c_gfa'])
print(len(filtered_bldgs_df3))



243
175
144
132


#### Make New Fields to Store Rankings

In [142]:
# Normalize the values of 'ec_per_occupant_a_to_c' and 'eci_a_to_c_gfa' on a scale of 0 to 1
filtered_bldgs_df3['norm_ec_per_occupant'] = (
    (filtered_bldgs_df3['ec_per_occupant_a_to_c'] - filtered_bldgs_df3['ec_per_occupant_a_to_c'].min()) /
    (filtered_bldgs_df3['ec_per_occupant_a_to_c'].max() - filtered_bldgs_df3['ec_per_occupant_a_to_c'].min())
)

filtered_bldgs_df3['norm_eci_gfa'] = (
    (filtered_bldgs_df3['eci_a_to_c_gfa'] - filtered_bldgs_df3['eci_a_to_c_gfa'].min()) /
    (filtered_bldgs_df3['eci_a_to_c_gfa'].max() - filtered_bldgs_df3['eci_a_to_c_gfa'].min())
)

# Calculate the shift in normalized carbon intensities
filtered_bldgs_df3['norm_shift'] = filtered_bldgs_df3['norm_ec_per_occupant'] - filtered_bldgs_df3['norm_eci_gfa']

In [143]:
# plot the list of unique values for bldg_prim_use along with the count of projects with that use
bldg_prim_use_counts = filtered_bldgs_df3['bldg_prim_use'].value_counts()
print(bldg_prim_use_counts)

# Remove any rows in the dataframe where the count of projects with that use is less than 5
filtered_bldgs_df3 = filtered_bldgs_df3[filtered_bldgs_df3['bldg_prim_use'].isin(bldg_prim_use_counts[bldg_prim_use_counts >= 5].index)]

bldg_prim_use
Education                                     41
Public Assembly                               23
Office                                        16
Residential: Multifamily (5 or more units)    13
Laboratory                                    10
Warehouse and Storage                          8
Healthcare                                     8
Public Order and Safety                        7
Transportation Hub                             2
Industrial                                     1
Lodging                                        1
Mercantile                                     1
Food Service                                   1
Name: count, dtype: int64


#### Make Normalize Shift Plot 1

In [162]:
# Prepare the data for the slope graph
projects = filtered_bldgs_df3.index  # Use the index to identify each project
occupant_norms = filtered_bldgs_df3['norm_ec_per_occupant']
gfa_norms = filtered_bldgs_df3['norm_eci_gfa']
primary_uses = filtered_bldgs_df3['bldg_prim_use']  # Get the primary use for each project

# Create a color mapping for 'bldg_prim_use'
unique_uses = primary_uses.unique()
color_map = {use: px.colors.qualitative.Prism[i % len(px.colors.qualitative.Prism)] for i, use in enumerate(unique_uses)}

# Create the slope graph
fig3 = go.Figure()

# Add lines connecting the two normalized values for each project
for project, occupant_norm, gfa_norm, primary_use in zip(projects, occupant_norms, gfa_norms, primary_uses):
    fig3.add_trace(go.Scatter(
        x=['norm_ec_per_occupant', 'norm_eci_gfa'],  # Two columns on the x-axis
        y=[occupant_norm, gfa_norm],  # Corresponding normalized values
        mode='lines+markers',  # Add lines and markers
        line=dict(color=color_map[primary_use], width=1),  # Color based on primary use
        marker=dict(size=6, color=color_map[primary_use]),  # Same color for markers
        hoverinfo='text',  # Show custom hover text
        text=[f"Project: {project}<br>Primary Use: {primary_use}<br>Normalized Value: {occupant_norm:.2f}",
              f"Project: {project}<br>Primary Use: {primary_use}<br>Normalized Value: {gfa_norm:.2f}"],  # Hover text for each point
        showlegend=False  # Disable legend for individual traces
    ))

# Add a legend for the primary uses
for use, color in color_map.items():
    fig3.add_trace(go.Scatter(
        x=[None], y=[None],  # Dummy points for legend
        mode='markers',
        marker=dict(size=10, color=color),
        name=use
    ))

# Customize the layout
fig3.update_layout(
    title={
        'text': 'Normalized Carbon Intensities By Occupant vs Area',  # Updated title
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 16, 'family': 'Arial', 'color': 'black', 'weight': 'bold'}
    },
    xaxis=dict(
        tickvals=['norm_ec_per_occupant', 'norm_eci_gfa'],  # Two columns
        ticktext=['By Occupant', 'By Area'],  # Custom labels
        showgrid=False,
        title_font=dict(size=12, family="Arial", color="black", weight="bold"),
    ),
    yaxis=dict(
        title='Normalized Carbon Intensity',
        range=[-0.01, 1.005],
        showgrid=True,
        gridcolor='lightgrey',
        title_font=dict(size=12, family="Arial", color="black", weight="bold"),

    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=True,
    height=1200,
    width=800
)

# Show the plot
fig3.show()

#### Make Normalize Shift Plot 2

In [160]:
fig4 = px.scatter(
    filtered_bldgs_df3,
    x='bldg_gfa',  # Total Building Area on the x-axis (m²)
    y='norm_shift',  # Normalized shift on the y-axis
    color='bldg_prim_use',  # Color based on 'bldg_prim_use'
    title='Delta of Normalized Intensity over Building Area',  # Updated title
    labels={
        'bldg_gfa': 'Building Area [m²]',
        'norm_shift': 'Normalized Carbon Intensity Shift (Occupant vs. Area)',
        'bldg_prim_use': 'Primary Use'
    },
    opacity=0.7,  # Add transparency to the dots
    color_discrete_sequence=px.colors.qualitative.Prism
)

# Add a horizontal line at y=0
fig4.add_hline(
    y=0,
    line_color='darkgrey',
    line_width=2,
    line_dash='dash'
)

# Customize the layout
# Customize the layout
fig4.update_layout(
    title={
        'text': 'Delta of Normalized Intensity over Building Area',
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 16, 'family': 'Arial', 'color': 'black', 'weight': 'bold'}
    },
    xaxis=dict(
        range=[0, 35000],  # Crop the x-axis to a maximum of 35,000 m²
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        range=[-1, 1],  # Set y-axis range from -1 to 1
        showgrid=True,
        gridcolor='lightgrey'
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=600,
    width=800,
    legend=dict(
        orientation='h',  # Set the legend to horizontal
        x=0.5,  # Center the legend horizontally
        y=-0.2,  # Position the legend below the plot
        xanchor='center',  # Anchor the legend horizontally at the center
        yanchor='top',  # Anchor the legend vertically at the top,
        title = None
    )
)

# Show the plot
fig4.show()

In [146]:
# Group by 'bldg_prim_use' and calculate the average 'norm_shift' and count of rows
summary_table = filtered_bldgs_df3.groupby('bldg_prim_use').agg(
    avg_norm_shift=('norm_shift', 'mean'),
    count=('norm_shift', 'size')
).reset_index()

# Round the average norm_shift to 2 decimal places for better readability
summary_table['avg_norm_shift'] = summary_table['avg_norm_shift'].round(2)

# Sort the table by 'avg_norm_shift'
summary_table = summary_table.sort_values(by='avg_norm_shift', ascending=True)

# Create a Plotly table
fig_table = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>Primary Use</b>', '<b>Average Intensity Delta</b>', '<b>Project Count</b>'],
        fill_color='lightgrey',
        align='left',
        font=dict(size=12, color='black')
    ),
    cells=dict(
        values=[
            summary_table['bldg_prim_use'],  # Primary Use
            summary_table['avg_norm_shift'],  # Average Norm Shift
            summary_table['count']  # Count
        ],
        fill_color='white',
        align='left',
        font=dict(size=12, color='black')
    )
)])

# Customize the layout
fig_table.update_layout(
    title='Normalized Intensity Delta (Occupant vs. Area) by Primary Use',
    title_x=0.5,  # Center the title
    height=500,
    width=600,
    paper_bgcolor='white',
    title_font=dict(size=16, family='Arial', color='black', weight='bold'),  # Make the title bold
)

# Show the table
fig_table.show()

---

### Exploring Differences Through Design Phase LCA is Conducted

#### Cleaning and Filtering the Data

In [147]:
# Remove any rows from the DataFrame where the bldg_stories_above field is NaN
filtered_bldgs_df4 = buildings_metadata_df.copy()

# Remove data points where the "eci_a_to_c_cfa" field is above 1200
filtered_bldgs_df4 = filtered_bldgs_df4[filtered_bldgs_df4['eci_a_to_c_cfa'] <= 1200]

# Limit to New Construction
filtered_bldgs_df4 = filtered_bldgs_df4[filtered_bldgs_df4['bldg_proj_type'] == 'New Construction']

# Count the data points in each bucket and reorder the list in the specified order
lca_phase_order = ['Design Development', 'Construction Documents', 'Construction', 'Completed']
phase_counts = filtered_bldgs_df4['lca_design_phase'].value_counts()
phase_counts = phase_counts.reindex(lca_phase_order)
print(phase_counts)



lca_design_phase
Design Development        17
Construction Documents    94
Construction              63
Completed                 66
Name: count, dtype: int64


#### Plotting the Data

In [148]:
# Create a box and whisker plot for the "eci_a_to_c_cfa" field grouped by "lca_design_phase"
fig5 = px.box(
    filtered_bldgs_df4,
    x='lca_design_phase',  # Group by LCA Design Phase
    y='eci_a_to_c_cfa',  # Plot this field
    title='Embodied Carbon Intensities Grouped by Design Phase',
    labels={
        'lca_design_phase': 'LCA Design Phase',
        'eci_a_to_c_cfa': 'Normalized Embodied Carbon Intensity (CFA)'
    },
    category_orders={'lca_design_phase': lca_phase_order},  # Ensure correct order of design phases
    hover_data=['bldg_prim_use']  # Include primary use in hover data
)

# Add all data points as dots with slight transparency and a different color
fig5.update_traces(
    boxpoints='all',
    jitter=0.3,
    pointpos=0,
    marker=dict(color='rgba(28, 144, 153, 0.5)'),
    fillcolor='rgb(236,226,240)',
    line=dict(color='rgb(166,189,219)')
)

# Customize the layout
fig5.update_layout(
    title={
        'text': 'Embodied Carbon Intensities Grouped by Design Phase',
        'x': 0.5,  # Center the title
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 16, 'family': 'Arial', 'color': 'black', 'weight': 'bold'}  # Make the title bold
    },
    xaxis=dict(
        title='LCA Design Phase',
        showgrid=False
    ),
    yaxis=dict(
        title='Normalized Embodied Carbon Intensity (CFA)',
        showgrid=True,
        gridcolor='lightgrey'
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=600,
    width=800
)

# Show the plot
fig5.show()

---

### Comparing Results in Tally vs. One Click

#### Bucketing the Data

In [149]:
# Remove any rows from the DataFrame where the bldg_stories_above field is NaN
filtered_bldgs_df5 = buildings_metadata_df.copy()

# Limit to New Construction
filtered_bldgs_df5 = filtered_bldgs_df5[filtered_bldgs_df5['bldg_proj_type'] == 'New Construction']

# Count the data points in each bucket
software_counts = filtered_bldgs_df5['lca_software'].value_counts()
print(software_counts)

lca_software
Tally LCA        159
One Click LCA     84
Name: count, dtype: int64


In [150]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Define the fields, their corresponding x-axis labels, and max value to cap chart at
fields_and_labels_and_caps = {
    'eci_a_to_c_cfa': ('Embodied Carbon Intensity [kgCO2e/m²]', 1200),
    'epi_a_to_c_cfa': ('Eutrophication Potential Intensity [kgNe/m²]', 1),
    'api_a_to_c_cfa': ('Acidification Potential Intensity [kgSO2e/m²]', 6.5),
    'sfpi_a_to_c_cfa': ('Smog Formation Potential Intensity [kgO3/m²]', 60),
    'odpi_a_to_c_cfa': ('Ozone Depletion Potential Intensity [kgCFC11e/m²]', 0.0001),
    'nredi_a_to_c_cfa': ('Non-renewable Energy Demand Intensity [MJ/m²]', 12000)
}

# Create a subplot figure
fig6 = make_subplots(
    rows=len(fields_and_labels_and_caps), cols=1,  # One row per field
    shared_yaxes=True,  # Share the y-axis for alignment
    vertical_spacing=0.08  # Space between plots
)

# Add a box and whisker plot for each field
for i, (field, (label, cap)) in enumerate(fields_and_labels_and_caps.items(), start=1):
    # Apply capping
    values = filtered_bldgs_df5[field].clip(upper=cap) if cap is not None else filtered_bldgs_df5[field]

    # Add the box plot to the subplot
    fig6.add_trace(
        go.Box(
            x=values,
            y=filtered_bldgs_df5['lca_software'],
            boxpoints='all',
            jitter=0.3,
            pointpos=0,  # Position points on the box
            marker=dict(color='rgba(28, 144, 153, 0.5)'),
            fillcolor='rgb(236,226,240)',
            line=dict(color='rgb(166,189,219)'),
            orientation='h',  # Horizontal orientation
            name=label  # Use the label as the trace name
        ),
        row=i, col=1  # Add to the appropriate subplot
    )

# Customize the layout
fig6.update_layout(
    title={
        'text': 'Comparison of Environmental Impact Intensities by LCA Software',
        'x': 0.5,  # Center the title
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 16, 'family': 'Arial', 'color': 'black', 'weight': 'bold'}  # Make the title bold
    },
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=250 * len(fields_and_labels_and_caps),  # Adjust height based on the number of plots
    width=800,
    showlegend=False
)

# Update x-axis titles, ranges, and gridlines for each subplot
for i, (field, (label, cap)) in enumerate(fields_and_labels_and_caps.items(), start=1):
    buffer = cap * 0.03 if cap is not None else 0  # Add 3% buffer to the cap so marks don't get cutoff
    fig6.update_xaxes(
        title_text=label,
        range=[0, cap + buffer] if cap is not None else None,  # Extend the range slightly
        showgrid=True,
        gridcolor='lightgrey',
        row=i, col=1
    )

# Show the figure
fig6.show()